In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

In [2]:
import pymongo

In [7]:
import yaml

In [17]:
with open(".passwords/mongo_pwd.yml", 'r') as stream:
    try:
        auth = yaml.load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [18]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % ('guest', auth['password']))
db = client["1-gram_time"]

In [19]:
df = pd.DataFrame(list(db["tweets"].find({ "word": "and" })))
df.head(5)

,_id,counts,rank,time,word
0,5c328d4a5920610636fb2ca0,2374312,8,2018-05-20,and
1,5c328d4d5920610636fcb3dc,2011354,9,2014-07-06,and
2,5c328d505920610636fe3b09,130232,9,2010-02-22,and
3,5c328d545920610636ffe431,1827301,9,2018-05-03,and
4,5c328d575920610636016df4,1917554,9,2015-02-17,and


In [20]:
def load(word):
    df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
    #df['date'] = [dt.strptime(time, '%Y-%m-%d') for time in df['time']]
    df['year'] = [date.year for date in df['time']]
    df['day'] = [date.timetuple().tm_yday for date in df['time']]
    return df

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Dropdown(
        id='my-dropdown',
        options = options,
        value=['@realdonaldtrump','@hillaryclinton','@barackobama'],
        multi=True
    ),
    dcc.Graph(id='my-graph'),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    data=[]
    figure = {'data':data}
    for item in selected_dropdown_value:
        df = load(item)
        data.append({'x':df['Date'], 'y':df['Rank'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server()